# Step 10 - Geodesic distances 

In [1]:
# These lines are necessary only if GemPy is not installed
import sys, os
sys.path.append("../../..")
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
os.environ['MKL_THREADING_LAYER'] = 'GNU'

# Importing GemPy, which takes really long
import gempy as gp

# Importing auxiliary libraries
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import matplotlib.tri as mtri
import pandas as pd
#import gdist

In [2]:
# Importing the data from CSV-files and setting extent and resolution, example without faults
geo_data = gp.create_data([0,3000,0,200,0,2000],resolution=[120,4,80], 
                         path_o = "C:/Users/Jan/gempy/notebooks/input_data/tut_chapter3/tutorial_ch3_foliations", # importing orientation (foliation) data
                         path_i = "C:/Users/Jan/gempy/notebooks/input_data/tut_chapter3/tutorial_ch3_interfaces") # importing point-positional interface data

In [3]:
interp_data = gp.InterpolatorData(geo_data, u_grade=[1], output='geology', compile_theano=True, theano_optimizer='fast_compile')

Compiling theano function...


INFO (theano.gof.compilelock): Waiting for existing lock by unknown process (I am process '3208')
INFO (theano.gof.compilelock): To manually release the lock, delete C:\Users\Jan\AppData\Local\Theano\compiledir_Windows-10-10.0.17134-SP0-Intel64_Family_6_Model_60_Stepping_3_GenuineIntel-3.6.4-64\lock_dir
WARNING (theano.tensor.blas): We did not find a dynamic library in the library_dir of the library we use for blas. If you use ATLAS, make sure to compile it with dynamics library.


Compilation Done!
Level of Optimization:  fast_compile
Device:  cpu
Precision:  float32
Number of faults:  0


In [4]:
lith_block, fault_block = gp.compute_model(interp_data)

In [5]:
print(lith_block.shape)

(2, 38400)


In [ ]:
ver , sim = gp.get_surfaces(interp_data, lith_block[1])

In [ ]:
gp.plotting.plot_surfaces_3D(geo_data, ver, sim)

In [ ]:
print(len(ver[3][:,0]))

In [ ]:
coord = np.vstack((ver[0], ver[1], ver[2], ver[3]))
print(coord.shape)
print(coord[:,0].shape)

In [ ]:
def set_axes_radius(ax, origin, radius):
    ax.set_xlim3d([origin[0] - radius, origin[0] + radius])
    ax.set_ylim3d([origin[1] - radius, origin[1] + radius])
    ax.set_zlim3d([origin[2] - radius, origin[2] + radius])

def set_axes_equal(ax):
    '''Make axes of 3D plot have equal scale so that spheres appear as spheres,
    cubes as cubes, etc..  This is one possible solution to Matplotlib's
    ax.set_aspect('equal') and ax.axis('equal') not working for 3D.

    Input
      ax: a matplotlib axis, e.g., as output from plt.gca().
    '''

    limits = np.array([
        ax.get_xlim3d(),
        ax.get_ylim3d(),
        ax.get_zlim3d(),
    ])

    origin = np.mean(limits, axis=1)
    radius = 0.5 * np.max(np.abs(limits[:, 1] - limits[:, 0]))
    set_axes_radius(ax, origin, radius)

In [ ]:
fig = plt.figure(figsize=(16,10))

# Plot the surface.  The triangles in parameter space determine which x, y, z
# points are connected by an edge.

ax = fig.add_subplot(1, 1, 1, projection='3d')
ax.plot_trisurf(ver[0][:,0], ver[0][:,1], ver[0][:,2], triangles=sim)
ax.plot_trisurf(ver[1][:,0], ver[1][:,1], ver[1][:,2], triangles=sim, cmap=plt.cm.Spectral)
ax.plot_trisurf(ver[2][:,0], ver[2][:,1], ver[2][:,2], triangles=sim, cmap=plt.cm.Spectral)
ax.plot_trisurf(ver[3][:,0], ver[3][:,1], ver[3][:,2], triangles=sim, cmap=plt.cm.Spectral)
set_axes_equal(ax) 
